## SFINCS - grpc4bmi example

This notebook shows how the SFINCS BMI server built in this package can be used
with grpc4bmi. It assumes you have built the docker container with

```
docker build -t sfincs-bmiserver .
```

### Requirements

We use grpc4bmi to communicate with the BMI model inside a container.

In [1]:
# pip installa grpc4bmi

### Case specification

SFINCS runs from within the working directory where the input files are located.
CD to the humber case dir

In [2]:
%cd /home/peter/ewatercycle/sfincs-bmi-server/cases/humber/sfincs_humber_executed
%ls

/home/peter/ewatercycle/sfincs-bmi-server/cases/humber/sfincs_humber_executed
figs/        log.txt     sfincs.bzs  sfincs.ind  sfincs.msk  sfincs.src
gis/         precip.nc   sfincs.dep  sfincs.inp  sfincs.obs  sfincs_his.nc
hydromt.log  sfincs.bnd  sfincs.dis  sfincs.man  sfincs.scs  sfincs_map.nc


The use example from https://github.com/Deltares/hydromt_sfincs repo as case specification.

In [ ]:
%git clone https://github.com/Deltares/hydromt_sfincs

In [1]:
%cd hydromt_sfincs/examples/sfincs_compound

/home/verhoes/git/eWaterCycle/sfincs-bmi-server/hydromt_sfincs/examples/sfincs_compound


### Instantiate the model

This starts the model container and connects the client to it.
We can already get the component name

In [2]:
from grpc4bmi.bmi_client_docker import BmiClientDocker
model = BmiClientDocker(image='sfincs-bmiserver', image_port=50051, work_dir="./")
model.get_component_name()

'Sfincs hydrodynamic model (C)'

### Interact with the model

We can initialize the model and see that it had effect: time variables seem to be set from the input file

In [3]:
model.initialize('sfincs.inp')
model.get_current_time(), model.get_start_time(), model.get_end_time()

(0.0, 0.0, 86400.0)

Updating the model should advance the model time. However, the initial timestep
is very small! The get_current_time value doesn't even seem to render to such
precision. This way it will take a million update steps just to advance the
model 1 time unit.

In [4]:
for i in range(10):
    print(model.get_current_time(), model.get_time_step())
    model.update()
    

0.0 9.999999974752427e-07
3.258591413497925 3.258591413497925
6.51718282699585 3.258591413497925
9.775774240493774 3.258591413497925
13.0343656539917 3.258591413497925
19.55154848098755 3.258591413497925
26.0687313079834 3.258591413497925
29.327322721481323 3.258591413497925
35.84450554847717 3.258591413497925
39.1030969619751 3.258591413497925


Let's try getting it past the first 10 time units...

In [5]:
t = model.get_current_time()
while t < model.get_start_time() + 10:
    model.update()
    t = model.get_current_time()

print(t)

42.36168837547302


In [6]:
print(model.get_current_time(), model.get_time_step())

42.36168837547302 3.258591413497925


In [7]:
model.get_output_var_names()

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Not Implemented"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:50029 {created_time:"2023-09-29T11:31:04.29291013+02:00", grpc_status:13, grpc_message:"Not Implemented"}"
>

After 10 minutes, nothing seems to have happened!

When I enforced the timestep to be 1 instead of `model.get_time_step()` (in the
c++ wrapper), the model actually did advance and the time step value also
changed to a more logical value (order of 1).